<a href="https://colab.research.google.com/github/mgfrantz/CalTech-CTME-AramCo-2025/blob/main/notebooks/03_serve_adapter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!uv pip install -qqq vllm litellm bitsandbytes

In [5]:
import os
from google.colab import userdata
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

Open a terminal 👇 and run th following commands:

1. Add your `HF_TOKEN` env variable (copy from secrets manager 👈)

```sh
export HF_TOKEN=<your_hf_token>
```

2. Serve the model with your adapter

```sh
vllm serve meta-llama/Llama-3.2-3B \
    --max-model-len 2048 \
    --enable-lora \
    --max-lora-rank 32 \
    --lora-modules sql-lora=mgfrantz/ctme-demo
```

In [6]:
from litellm import completion

In [19]:
vllm_url = 'http://localhost:8000/v1/'
model = 'openai/sql-lora'

In [38]:
messages = [
    {
      "role": "system",
      "content": "You are an expert SQL developer specializing in SQLite. Generate accurate SQL queries that follow these requirements:\n\nSQLITE REQUIREMENTS:\n1. Use only standard SQLite-compatible SQL syntax\n2. Avoid PostgreSQL-specific functions like INTERVAL - use date arithmetic instead\n3. Use SQLite date functions: date(), datetime(), julianday() for date calculations\n4. For \"recent month\" use: WHERE date_column >= date((SELECT MAX(date_column) FROM table_name), '-1 month')\n5. For \"past 30 days\" use: WHERE date_column >= date((SELECT MAX(date_column) FROM table_name), '-30 days')\n6. For rolling averages, use window functions or subqueries\n7. Use MAX(appropriate_date_column) to find the most recent date\n8. Use proper SQLite column types: INTEGER, TEXT, REAL, BLOB\n9. Handle foreign key relationships correctly with proper JOIN syntax\n\nQUERY STANDARDS:\n- Write clear, efficient queries\n- Use appropriate aggregation functions (COUNT, SUM, AVG, MAX, MIN)\n- Include proper GROUP BY and ORDER BY clauses when needed\n- Use table aliases for readability in complex queries"
    },
    {
      "role": "user",
      "content": "Database Schema:\nTable: customers\n  - id (BIGINT, PRIMARY KEY, NOT NULL)\n  - first_name (VARCHAR, NOT NULL)\n  - last_name (VARCHAR, NOT NULL)\n  - email (VARCHAR, NOT NULL)\n  - phone_number (VARCHAR, NOT NULL)\n\nTable: drivers\n  - id (BIGINT, PRIMARY KEY, NOT NULL)\n  - first_name (VARCHAR, NOT NULL)\n  - last_name (VARCHAR, NOT NULL)\n  - phone_number (VARCHAR, NOT NULL)\n  - license_number (VARCHAR, NOT NULL)\n\nTable: orders\n  - id (BIGINT, PRIMARY KEY, NOT NULL)\n  - customer_id (BIGINT, NOT NULL)\n  - restaurant_id (BIGINT, NOT NULL)\n  - order_date (TIMESTAMP, NOT NULL)\n  - total_amount (DOUBLE, NOT NULL)\n  - status (VARCHAR, NOT NULL)\n\nTable: restaurants\n  - id (BIGINT, PRIMARY KEY, NOT NULL)\n  - name (VARCHAR, NOT NULL)\n  - address (VARCHAR, NOT NULL)\n  - phone_number (VARCHAR, NOT NULL)\n\nQuestion: How many orders were placed in the most recent month?"
    },
    # {
    #   "role": "assistant",
    #   "content": "SELECT COUNT(*) FROM orders WHERE order_date >= date((SELECT MAX(order_date) FROM orders), '-1 month')"
    # }
]

In [40]:
resp = completion(
    model,
    messages,
    base_url=vllm_url,
    api_key='not-needed',
    max_completion_tokens=128
)

/usr/local/lib/python3.11/dist-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content="Solution...ields={'refusal': None}), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...{'stop_reason': 128001}), input_type=Choices])
  return self.__pydantic_serializer__.to_python(


In [41]:
print(resp.choices[0].message.content)

Solution: The following SQL query can be used to find the number of orders placed in the most recent month:

SELECT COUNT(*) AS num_orders
FROM orders
WHERE order_date >= date((SELECT MAX(order_date) FROM orders), '-1 month');

This query uses the SQLite date() function to extract the most recent month from the order_date column and then compares it against the current date. It then counts the number of orders placed during this period using the COUNT() function.
